In [2]:
import cutlass               
import cutlass.cute as cute  
from cutlass.cutlass_dsl import (
    Boolean,
    Integer,
    Int32,
    min,
    extract_mlir_values,
    new_from_mlir_values,
    dsl_user_op,
)


@cute.jit
def print_layout_2d(L: cute.Tensor):
    m, n = L.shape
    for i in cute.range_constexpr(m):
        for j in cute.range_constexpr(n):
            cute.printf("{} ", L((i, j)), end="")
        cute.printf("")
    cute.printf("")
@cute.jit
def layout_function_example():
    """
    Layout function in cutlass
    """
    S = (2, 4)
    D = (2, 2)
    L = cute.make_layout(shape=S, stride=D)
    print_layout_2d(L)


    S = (cute.size(2), 4)
    D = (4, 1)
    L = cute.make_layout(shape=S, stride=D)
    cute.printf("row_matrix for {}", L)
    print_layout_2d(L)

    S = (2, 4)
    D = (1, 2)
    L = cute.make_layout(shape=S, stride=D)
    cute.printf("col_matrix for {}", L)
    print_layout_2d(L)

layout_function_example()

0 2 4 6 
2 4 6 8 

row_matrix for (2,4):(4,1)
0 1 2 3 
4 5 6 7 

col_matrix for (2,4):(1,2)
0 2 4 6 
1 3 5 7 



In [3]:
@cute.jit 
def create_tensor_from_ptr(ptr: cute.Pointer):
    layout = cute.make_layout((8, 5), stride=(5, 1))
    tensor = cute.make_tensor(ptr, layout)
    tensor.fill(1)
    cute.print_tensor(tensor)    
    print_layout_2d(layout)


import torch

from cutlass.torch import dtype as torch_dtype
import cutlass.cute.runtime as cute_rt

a = torch.randn(8, 5, dtype=torch_dtype(cutlass.Float32))
ptr_a = cute_rt.make_ptr(cutlass.Float32, a.data_ptr())

create_tensor_from_ptr(ptr_a)

tensor(raw_ptr(0x000000000a16c280: f32, generic, align<4>) o (8,5):(5,1), data=
       [[ 1.000000,  1.000000,  1.000000,  1.000000,  1.000000, ],
        [ 1.000000,  1.000000,  1.000000,  1.000000,  1.000000, ],
        [ 1.000000,  1.000000,  1.000000,  1.000000,  1.000000, ],
        ...
        [ 1.000000,  1.000000,  1.000000,  1.000000,  1.000000, ],
        [ 1.000000,  1.000000,  1.000000,  1.000000,  1.000000, ],
        [ 1.000000,  1.000000,  1.000000,  1.000000,  1.000000, ]])
0 1 2 3 4 
5 6 7 8 9 
10 11 12 13 14 
15 16 17 18 19 
20 21 22 23 24 
25 26 27 28 29 
30 31 32 33 34 
35 36 37 38 39 



In [4]:
from cutlass.cute.runtime import from_dlpack

@cute.jit
def print_tensor_dlpack(src: cute.Tensor):
    print("raw_print", src)
    cute.print_tensor(src)

a = torch.randn(8, 5, dtype=torch_dtype(cutlass.Float32))

print_tensor_dlpack(from_dlpack(a))

raw_print tensor<ptr<f32, generic> o (8,5):(5,1)>
tensor(raw_ptr(0x000000000a8c9a00: f32, generic, align<4>) o (8,5):(5,1), data=
       [[-1.664767,  0.477034, -0.459961, -0.290820,  1.432731, ],
        [ 0.233584,  1.070363,  0.215888,  0.656703, -2.543746, ],
        [ 0.745666,  1.367355, -1.577380,  3.179004, -0.739842, ],
        ...
        [ 0.754863, -0.378220,  1.022583, -0.854756,  0.438261, ],
        [ 0.310660,  0.757676,  0.591205, -0.029650,  1.128956, ],
        [ 0.453425,  0.042322,  2.021724,  1.389728, -0.443380, ]])


In [5]:
import numpy as np

a = np.random.randn(8, 8).astype(np.float32)

print_tensor_dlpack(from_dlpack(a))

raw_print tensor<ptr<f32, generic> o (8,8):(8,1)>
tensor(raw_ptr(0x000000000a42fbd0: f32, generic, align<4>) o (8,8):(8,1), data=
       [[-0.819895, -0.366917,  0.101130, ..., -1.124290, -0.972739,  0.144119, ],
        [ 0.809396,  1.097854,  0.008393, ...,  0.345199,  0.035947,  0.395983, ],
        [ 1.823589, -1.152929,  0.142579, ..., -0.215565, -1.010382,  1.037992, ],
        ...
        [ 0.709613, -0.633216, -0.725405, ..., -0.390990, -0.343865, -0.089471, ],
        [-0.512715, -1.168170,  0.968349, ...,  0.703410,  1.750636,  2.296905, ],
        [ 0.015170, -0.832358,  0.891237, ...,  0.461414, -1.039721,  0.001884, ]])


In [6]:
@cute.jit
def tensor_access_item(a: cute.Tensor):
    # access data using linear index
    cute.printf("a[2] = {} (equivalent to a[{}])", a[2],
                cute.make_identity_tensor(a.layout.shape)[2])
    cute.printf("a[9] = {} (equivalent to a[{}])", a[9],
                cute.make_identity_tensor(a.layout.shape)[9])

    # access data using n-d coordinates, following two are equivalent
    cute.printf("a[2,0] = {}", a[2, 0])
    cute.printf("a[2,4] = {}", a[2, 4])
    cute.printf("a[(2,4)] = {}", a[2, 4])
    cute.printf("a[(2,:)] = {}", a[1, None])
    print_tensor_2d(a)
    # cute.printf("a = {}", a)

    # assign value to tensor@(2,4)
    a[2,3] = 100.0
    a[2,4] = 101.0
    cute.printf("a[2,3] = {}", a[2,3])
    cute.printf("a[(2,4)] = {}", a[(2,4)])

@cute.kernel
def print_tensor_gpu(ptr: cute.Pointer):
    layout = cute.make_layout((8, 5), stride=(5, 1))
    tensor = cute.make_tensor(ptr, layout)

    tidx, _, _ = cute.arch.thread_idx()

    if tidx == 0:
        cute.print_tensor(tensor)


# Create a tensor with sequential data using torch
data = torch.arange(0, 8*5, dtype=torch.float32).reshape(8, 5)
print("data.ptr", hex(data.data_ptr()))
tensor_access_item(from_dlpack(data))

print(data)

data.ptr 0xb350880


DSLRuntimeError: DSLRuntimeError: 💥💥💥 Error during runtime code generation for function `tensor_access_item` 💥💥💥
  Caused exception: name 'print_tensor_2d' is not defined
[92m💡 Suggestions:[0m
 Using variables defined in dynamic control flow is not supported. Please give an initial value before control flow.

In [ ]:
@cute.jit
def print_tensor_2d(L: cute.Tensor):
    m, n = L.shape
    for i in cute.range_constexpr(m):
        for j in cute.range_constexpr(n):
            if j != 0: 
                cute.printf(", ", end="")
            cute.printf("{}", L[(i, j)], end="")
        cute.printf("")
    cute.printf("")
    

@cute.jit
def print_tensor_coord(a: cute.Tensor):
    coord_tensor = cute.make_identity_tensor(a.layout.shape)
    print(coord_tensor)
    print_tensor_2d(coord_tensor)

a = torch.randn(8,4, dtype=torch_dtype(cutlass.Float32))
print_tensor_coord(from_dlpack(a))

tensor<(0,0) o (8,4):(1@0,1@1)>
(0,0), (0,1), (0,2), (0,3)
(1,0), (1,1), (1,2), (1,3)
(2,0), (2,1), (2,2), (2,3)
(3,0), (3,1), (3,2), (3,3)
(4,0), (4,1), (4,2), (4,3)
(5,0), (5,1), (5,2), (5,3)
(6,0), (6,1), (6,2), (6,3)
(7,0), (7,1), (7,2), (7,3)



# TensorSSA
You can think of it as a tensor residing in a (simulated) register.

By overloading numerous Python operators, it allows users to express tensor computations (primarily element-wise operations and reductions) in a more Pythonic way. These element-wise operations are then translated into `optimized vectorization instructions.`

In [ ]:
@cute.jit
def load_and_store(res: cute.Tensor, a: cute.Tensor, b: cute.Tensor):
    """
    Load data from memory and store the result to memory.

    :param res: The destination tensor to store the result.
    :param a: The source tensor to be loaded.
    :param b: The source tensor to be loaded.
    """
    a_vec = a.load()
    cute.print_tensor(a)
    print(f"a_vec: {a_vec}")      # prints `a_vec: vector<12xf32> o (3, 4)`
    b_vec = b.load()
    print(f"b_vec: {b_vec}")      # prints `b_vec: vector<12xf32> o (3, 4)`
    res.store(a_vec + b_vec)
    cute.print_tensor(res)

a = np.ones(12).reshape((3, 4)).astype(np.float32)
b = np.ones(12).reshape((3, 4)).astype(np.float32)
c = np.zeros(12).reshape((3, 4)).astype(np.float32)
load_and_store(from_dlpack(c), from_dlpack(a), from_dlpack(b))

a_vec: tensor_value<vector<12xf32> o (3, 4)>
b_vec: tensor_value<vector<12xf32> o (3, 4)>
tensor(raw_ptr(0x0000000010fe6d00: f32, generic, align<4>) o (3,4):(4,1), data=
       [[ 1.000000,  1.000000,  1.000000,  1.000000, ],
        [ 1.000000,  1.000000,  1.000000,  1.000000, ],
        [ 1.000000,  1.000000,  1.000000,  1.000000, ]])
tensor(raw_ptr(0x00000000118f9a50: f32, generic, align<4>) o (3,4):(4,1), data=
       [[ 2.000000,  2.000000,  2.000000,  2.000000, ],
        [ 2.000000,  2.000000,  2.000000,  2.000000, ],
        [ 2.000000,  2.000000,  2.000000,  2.000000, ]])


### Register-Level Tensor Operations

When writing kernel logic, various computations, transformations, slicing, etc., are performed on data loaded into registers.

In [ ]:
@cute.jit
def apply_slice(src: cute.Tensor, dst: cute.Tensor, indices: cutlass.Constexpr):
    """
    Apply slice operation on the src tensor and store the result to the dst tensor.

    :param src: The source tensor to be sliced.
    :param dst: The destination tensor to store the result.
    :param indices: The indices to slice the source tensor.
    """
    src_vec = src.load()
    dst_vec = src_vec[indices]
    print(f"{src_vec} -> {dst_vec}, {indices}")
    print(f"{type(dst_vec)}")
    cute.print_tensor(dst)
    dst.store(dst_vec)
    cute.print_tensor(dst)
    if isinstance(dst_vec, cute.core.TensorSSA):
        dst.store(dst_vec)
        cute.print_tensor(dst)
    else:
        dst[0] = dst_vec
        cute.print_tensor(dst)

def slice_1():
    src_shape = (4, 2, 3)
    dst_shape = (4, 3)
    indices = (None, 1, None)

    """
    a:
    [[[ 0.  1.  2.]
      [ 3.  4.  5.]]

     [[ 6.  7.  8.]
      [ 9. 10. 11.]]

     [[12. 13. 14.]
      [15. 16. 17.]]

     [[18. 19. 20.]
      [21. 22. 23.]]]
    """
    a = np.arange(np.prod(src_shape)).reshape(*src_shape).astype(np.float32)
    dst = np.random.randn(*dst_shape).astype(np.float32)
    apply_slice(from_dlpack(a), from_dlpack(dst), indices)

slice_1()

tensor_value<vector<24xf32> o (4, 2, 3)> -> tensor_value<vector<12xf32> o (4, 3)>, (None, 1, None)
<class 'cutlass.cute.core.TensorSSA'>


DSLRuntimeError: DSLRuntimeError: Unsupported DSL type: vector<12xf32>

In [ ]:
def slice_2():
    src_shape = (4, 2, 3)
    dst_shape = (1,)
    indices = 10
    a = np.arange(np.prod(src_shape)).reshape(*src_shape).astype(np.float32)
    dst = np.random.randn(*dst_shape).astype(np.float32)
    apply_slice(from_dlpack(a), from_dlpack(dst), indices)

slice_2()

tensor_value<vector<24xf32> o (4, 2, 3)> -> ?, 10
Float32


DSLRuntimeError: DSLRuntimeError: 💥💥💥 Error during runtime code generation for function `apply_slice` 💥💥💥

In [ ]:
@cute.jit
def binary_op_1(res: cute.Tensor, a: cute.Tensor, b: cute.Tensor):
    a_vec = a.load()
    b_vec = b.load()

    add_res = a_vec + b_vec
    res.store(add_res)
    cute.print_tensor(res)        # prints [3.000000, 3.000000, 3.000000]

    sub_res = a_vec - b_vec
    res.store(sub_res)
    cute.print_tensor(res)        # prints [-1.000000, -1.000000, -1.000000]

    mul_res = a_vec * b_vec
    res.store(mul_res)
    cute.print_tensor(res)        # prints [2.000000, 2.000000, 2.000000]

    div_res = a_vec / b_vec
    res.store(div_res)
    cute.print_tensor(res)        # prints [0.500000, 0.500000, 0.500000]

    floor_div_res = a_vec // b_vec
    res.store(floor_div_res)
    cute.print_tensor(res)        # prints [0.000000, 0.000000, 0.000000]

    mod_res = a_vec % b_vec
    res.store(mod_res)
    cute.print_tensor(res)        # prints [1.000000, 1.000000, 1.000000]


a = np.empty((3,), dtype=np.float32)
a.fill(1.0)
b = np.empty((3,), dtype=np.float32)
b.fill(2.0)
res = np.empty((3,), dtype=np.float32)
binary_op_1(from_dlpack(res), from_dlpack(a), from_dlpack(b))

tensor(raw_ptr(0x000000000e502760: f32, generic, align<4>) o (3):(1), data=
       [ 3.000000, ],
       [ 3.000000, ],
       [ 3.000000, ])
tensor(raw_ptr(0x000000000e502760: f32, generic, align<4>) o (3):(1), data=
       [-1.000000, ],
       [-1.000000, ],
       [-1.000000, ])
tensor(raw_ptr(0x000000000e502760: f32, generic, align<4>) o (3):(1), data=
       [ 2.000000, ],
       [ 2.000000, ],
       [ 2.000000, ])
tensor(raw_ptr(0x000000000e502760: f32, generic, align<4>) o (3):(1), data=
       [ 0.500000, ],
       [ 0.500000, ],
       [ 0.500000, ])
tensor(raw_ptr(0x000000000e502760: f32, generic, align<4>) o (3):(1), data=
       [ 0.000000, ],
       [ 0.000000, ],
       [ 0.000000, ])
tensor(raw_ptr(0x000000000e502760: f32, generic, align<4>) o (3):(1), data=
       [ 1.000000, ],
       [ 1.000000, ],
       [ 1.000000, ])


In [ ]:
@cute.jit
def unary_op_1(res: cute.Tensor, a: cute.Tensor):
    a_vec = a.load()

    sqrt_res = cute.math.sqrt(a_vec)
    res.store(sqrt_res)
    cute.print_tensor(res)        # prints [2.000000, 2.000000, 2.000000]

    sin_res = cute.math.sin(a_vec)
    res.store(sin_res)
    cute.print_tensor(res)        # prints [-0.756802, -0.756802, -0.756802]

    exp2_res = cute.math.exp2(a_vec)
    res.store(exp2_res)
    cute.print_tensor(res)        # prints [16.000000, 16.000000, 16.000000]

a = np.array([4.0, 4.0, 4.0], dtype=np.float32)
res = np.empty((3,), dtype=np.float32)
unary_op_1(from_dlpack(res), from_dlpack(a))

tensor(raw_ptr(0x000000000e502760: f32, generic, align<4>) o (3):(1), data=
       [ 2.000000, ],
       [ 2.000000, ],
       [ 2.000000, ])
tensor(raw_ptr(0x000000000e502760: f32, generic, align<4>) o (3):(1), data=
       [-0.756802, ],
       [-0.756802, ],
       [-0.756802, ])
tensor(raw_ptr(0x000000000e502760: f32, generic, align<4>) o (3):(1), data=
       [ 16.000000, ],
       [ 16.000000, ],
       [ 16.000000, ])


#### Reduction Operation

The `TensorSSA`'s `reduce` method applies a specified reduction operation (`ReductionOp.ADD`, `ReductionOp.MUL`, `ReductionOp.MAX`, `ReductionOp.MIN`) starting with an initial value, and performs this reduction along the dimensions specified by the `reduction_profile.`. The result is typically a new `TensorSSA` with reduced dimensions or a scalar value if reduces across all axes.

In [ ]:
@cute.jit
def reduction_op(a: cute.Tensor):
    """
    Apply reduction operation on the src tensor.

    :param src: The source tensor to be reduced.
    """
    a_vec: "cute.TensorSSA" = a.load()
    red_res = a_vec.reduce(
        cute.ReductionOp.ADD,
        0.0,
        reduction_profile=0
    )
    cute.printf(red_res)        # prints 21.000000

    red_res = a_vec.reduce(
        cute.ReductionOp.ADD,
        0.0,
        reduction_profile=(None, 1)
    )
    # We can't print the TensorSSA directly at this point, so we store it to a new Tensor and print it.
    res = cute.make_fragment(red_res.shape, cutlass.Float32)
    res.store(red_res)
    cute.print_tensor(res)        # prints [6.000000, 15.000000]

    red_res = a_vec.reduce(
        cute.ReductionOp.ADD,
        1.0,
        reduction_profile=(1, None)
    )
    res = cute.make_fragment(red_res.shape, cutlass.Float32)
    res.store(red_res)
    cute.print_tensor(res)        # prints [6.000000, 8.000000, 10.000000]


a = np.array([[1, 2, 3], [4, 5, 6]], dtype=np.float32)
reduction_op(from_dlpack(a))

21.000000
tensor(raw_ptr(0x00007ffd65e86280: f32, rmem, align<32>) o (2):(1), data=
       [ 6.000000, ],
       [ 15.000000, ])
tensor(raw_ptr(0x00007ffd65e862a0: f32, rmem, align<32>) o (3):(1), data=
       [ 6.000000, ],
       [ 8.000000, ],
       [ 10.000000, ])


In [ ]:
@cute.jit
def coalesce_example():
    """
    Demonstrates coalesce operation flattening and combining modes
    """
    # layout = cute.make_layout((2, (2, 8)), stride=(1, (cutlass.Int32(2), 8))) # Dynamic stride
    layout = cute.make_layout(
        shape=(3, (2, 4)), 
        stride=(8, (4, 1))) # Dynamic stride
    # layout = cute.make_layout((4, 3), stride=(3, 1)) # Dynamic stride
    result = cute.coalesce(layout)
    m, (n, k) = layout.shape
    for i in cutlass.range_constexpr(m):
        for j in cutlass.range_constexpr(n):
            for k in cutlass.range_constexpr(k):
                cute.printf("fL{} = {}", (i, (j, k)), layout((i, (j, k))))
    for i in cutlass.range_constexpr(cute.size(layout)):
        cute.printf("fL({}) = {}", i, layout(i))
    cute.printf("Original: \t{}", layout)
    cute.printf("Coalesced: \t{}", result)
    cute.printf("depth of layout: {}", cute.depth(layout))
    cute.printf("depth of coalesced layout: {}", cute.depth(result))

coalesce_example()

fL(0,(0,0)) = 0
fL(0,(0,1)) = 1
fL(0,(0,2)) = 2
fL(0,(0,3)) = 3
fL(0,(1,0)) = 4
fL(0,(1,1)) = 5
fL(0,(1,2)) = 6
fL(1,(0,0)) = 8
fL(1,(0,1)) = 9
fL(1,(1,0)) = 12
fL(0) = 0
fL(1) = 8
fL(2) = 16
fL(3) = 4
fL(4) = 12
fL(5) = 20
fL(6) = 1
fL(7) = 9
fL(8) = 17
fL(9) = 5
fL(10) = 13
fL(11) = 21
fL(12) = 2
fL(13) = 10
fL(14) = 18
fL(15) = 6
fL(16) = 14
fL(17) = 22
fL(18) = 3
fL(19) = 11
fL(20) = 19
fL(21) = 7
fL(22) = 15
fL(23) = 23
Original: 	(3,(2,4)):(8,(4,1))
Coalesced: 	(3,2,4):(8,4,1)
depth of layout: 2
depth of coalesced layout: 1


In [ ]:
@cute.jit
def coalesce_post_conditions():
    """
    Demonstrates coalesce operation's 3 post-conditions:
    1. size(@a result) == size(@a layout)
    2. depth(@a result) <= 1
    3. for all i, 0 <= i < size(@a layout), @a result(i) == @a layout(i)
    """
    layout = cute.make_layout(
        ((2, (3, 4)), (3, 2), 1),
        stride=((4, (8, 24)), (2, 6), 12)
    )
    result = cute.coalesce(layout)

    print(">>> Original:", layout)
    print(">>> Coalesced:", result)

    print(">>> Checking post-conditions:")
    print(">>> 1. Checking size remains the same after the coalesce operation:")
    original_size = cute.size(layout)
    coalesced_size = cute.size(result)
    print(f"Original size: {original_size}, Coalesced size: {coalesced_size}")
    assert coalesced_size == original_size, \
            f"Size mismatch: original {original_size}, coalesced {coalesced_size}"
    
    print(">>> 2. Checking depth of coalesced layout <= 1:")
    depth = cute.depth(result)
    print(f"Depth of coalesced layout: {depth}")
    assert depth <= 1, f"Depth of coalesced layout should be <= 1, got {depth}"

    print(">>> 3. Checking layout functionality remains the same after the coalesce operation:")
    for i in cutlass.range_constexpr(original_size):
        original_value = layout(i)
        coalesced_value = result(i)
        print(f"Index {i}: original {original_value}, coalesced {coalesced_value}")
        assert coalesced_value == original_value, \
            f"Value mismatch at index {i}: original {original_value}, coalesced {coalesced_value}"

coalesce_post_conditions()

>>> Original: ((2,(3,4)),(3,2),1):((4,(8,24)),(2,6),12)
>>> Coalesced: (24,6):(4,2)
>>> Checking post-conditions:
>>> 1. Checking size remains the same after the coalesce operation:
Original size: 144, Coalesced size: 144
>>> 2. Checking depth of coalesced layout <= 1:
Depth of coalesced layout: 1
>>> 3. Checking layout functionality remains the same after the coalesce operation:
Index 0: original 0, coalesced 0
Index 1: original 4, coalesced 4
Index 2: original 8, coalesced 8
Index 3: original 12, coalesced 12
Index 4: original 16, coalesced 16
Index 5: original 20, coalesced 20
Index 6: original 24, coalesced 24
Index 7: original 28, coalesced 28
Index 8: original 32, coalesced 32
Index 9: original 36, coalesced 36
Index 10: original 40, coalesced 40
Index 11: original 44, coalesced 44
Index 12: original 48, coalesced 48
Index 13: original 52, coalesced 52
Index 14: original 56, coalesced 56
Index 15: original 60, coalesced 60
Index 16: original 64, coalesced 64
Index 17: original 68

/tmp/ipykernel_1788377/2625281346.py:32: UserWarning: This static loop has 144 iterations, which may be very slow to compile, consider using `cutlass.range(..., unroll_full=True)` instead.
  for i in cutlass.range_constexpr(original_size):


In [ ]:
@cute.jit
def bymode_coalesce_example():
    """
    Demonstrates by-mode coalescing
    """
    layout = cute.make_layout((2, (1, 6)), stride=(1, (6, 2)))

    # Coalesce with mode-wise profile (1,1) = coalesce both modes
    result = cute.coalesce(layout, target_profile=(1, 1))
    
    # Print results
    print(">>> Original: ", layout)
    print(">>> Coalesced Result: ", result)

bymode_coalesce_example()

>>> Original:  (2,(1,6)):(1,(6,2))
>>> Coalesced Result:  (2,6):(1,2)


In [ ]:
@cute.jit
def composition_example():
    """
    Demonstrates basic layout composition R = A ◦ B
    """
    A = cute.make_layout((6, 2), stride=(cutlass.Int32(8), 2)) # Dynamic stride
    B = cute.make_layout((4, 3), stride=(3, 1))
    R = cute.composition(A, B)

    # Print static and dynamic information
    # print(">>> Layout A:", A)
    cute.printf("Layout A: {}", A)
    # print(">>> Layout B:", B) 
    cute.printf("Layout B: {}", B)
    # print(">>> Composition R = A ◦ B:", R)
    cute.printf("Composition R: {}", R)
    
    for i in cutlass.range_constexpr(cute.size(R)):
        cute.printf("R({}) = {}, {}", i, R(i), A(B(i)))

composition_example()





Layout A: (6,2):(8,2)
Layout B: (4,3):(3,1)
Composition R: ((2,2),3):((24,2),8)
R(0) = 0, 0
R(1) = 24, 24
R(2) = 2, 2
R(3) = 26, 26
R(4) = 8, 8
R(5) = 32, 32
R(6) = 10, 10
R(7) = 34, 34
R(8) = 16, 16
R(9) = 40, 40
R(10) = 18, 18
R(11) = 42, 42


In [ ]:
@cute.jit
def composition_example():
    A= cute.make_layout((6, 2), stride=(8, 2))
    A = cute.coalesce(A)
    cute.printf("Layout A: {}", A)
    B = cute.make_layout((4, 3), stride=(3, 1))
    R = cute.composition(A, B)

    cute.printf("Layout B: {}", B)
    cute.printf("Composition R: {}", R)
composition_example()














Layout A: (6,2):(8,2)
Layout B: (4,3):(3,1)
Composition R: ((2,2),3):((24,2),8)


In [ ]:
@cute.jit
def composition_static_vs_dynamic_layout():
    """
    Shows difference between static and dynamic composition results
    """
    # Static version - using compile-time values
    A_static = cute.make_layout(
        (10, 2), 
        stride=(16, 4)
    )
    B_static = cute.make_layout(
        (5, 4), 
        stride=(1, 5)
    )
    R_static = cute.composition(A_static, B_static)

    # Static print shows compile-time info
    print(">>> Static composition:")
    print(">>> A_static: ", A_static)
    print(">>> B_static: ", B_static)
    print(">>> R_static: ", R_static)

    # Dynamic version - using runtime Int32 values
    A_dynamic = cute.make_layout(
        (cutlass.Int32(10), cutlass.Int32(2)),
        stride=(cutlass.Int32(16), cutlass.Int32(4))
    )
    B_dynamic = cute.make_layout(
        (cutlass.Int32(5), cutlass.Int32(4)),
        stride=(cutlass.Int32(1), cutlass.Int32(5))
    )
    R_dynamic = cute.composition(A_dynamic, B_dynamic)
    
    # Dynamic printf shows runtime values
    cute.printf(">?? Dynamic composition:")
    cute.printf(">?? A_dynamic: {}", A_dynamic)
    cute.printf(">?? B_dynamic: {}", B_dynamic)
    cute.printf(">?? R_dynamic: {}", R_dynamic)

composition_static_vs_dynamic_layout()

>>> Static composition:
>>> A_static:  (10,2):(16,4)
>>> B_static:  (5,4):(1,5)
>>> R_static:  (5,(2,2)):(16,(80,4))
>?? Dynamic composition:
>?? A_dynamic: (10,2):(16,4)
>?? B_dynamic: (5,4):(1,5)
>?? R_dynamic: ((5,1),(2,2)):((16,4),(80,4))


In [ ]:
@cute.jit
def bymode_composition_example():
    """
    Demonstrates by-mode composition using a tiler
    """
    # Define the original layout A
    A = cute.make_layout(
        (cutlass.Int32(12), (cutlass.Int32(4), cutlass.Int32(8))), 
        stride=(cutlass.Int32(59), (cutlass.Int32(13), cutlass.Int32(1)))
    )

    # Define the tiler for by-mode composition
    tiler = (3, 8) # Apply 3:1 to mode-0 and 8:1 to mode-1

    # Apply by-mode composition
    result = cute.composition(A, tiler)

    # Print static and dynamic information
    print(">>> Layout A:", A)
    cute.printf(">?? Layout A: {}", A)
    print(">>> Tiler:", tiler)
    cute.printf(">?? Tiler: {}", tiler)
    print(">>> By-mode Composition Result:", result)
    cute.printf(">?? By-mode Composition Result: {}", result)

bymode_composition_example()

In [ ]:
import torch
torch.utils.checkpoint.create_selective_checkpoint_contexts

AttributeError: module 'torch.utils' has no attribute 'checkpoint'

In [8]:
@cute.jit
def print_layout_2d(L: cute.Tensor):
    m, n = L.shape
    m = cute.size(L, [0])
    n = cute.size(L, [1])
    cute.printf("Layout: {}", L)
    for i in cute.range_constexpr(m):
        for j in cute.range_constexpr(n):
            cute.printf("%3d ", L((i, j)), end="")
        cute.printf("")
    cute.printf("")
@cute.jit
def composition_example():
    """
    Demonstrates basic layout composition R = A ◦ B
    """
    A = cute.make_layout((6, 2), stride=(cutlass.Int32(8), 2)) # Dynamic stride
    B = cute.make_layout((4, 3), stride=(3, 1))
    R = cute.composition(A, B)

    print_layout_2d(A) 
    print_layout_2d(B)
    print_layout_2d(R)
    #  (6, 2):(8, 2) o (4, 3):(3, 1)
    # -> (6, 2) o (4, 3):(3, 1) -> (2, 2):()
    # -> 
composition_example()





Layout: (6,2):(8,2)
  0   2 
  8  10 
 16  18 
 24  26 
 32  34 
 40  42 

Layout: (4,3):(3,1)
  0   1   2 
  3   4   5 
  6   7   8 
  9  10  11 

Layout: ((2,2),3):((24,2),8)
  0   8  16 
 24  32  40 
  2  10  18 
 26  34  42 



In [ ]:
@cute.jit
def composition_example():
    """
    Demonstrates basic layout composition R = A ◦ B
    """
    A = cute.make_layout((6, 2), stride=(cutlass.Int32(8), 2)) # Dynamic stride
    B = cute.make_layout((4, 3), stride=(3, 1))
    R = cute.composition(A, B)

    print_layout_2d(A) 
    print_layout_2d(B)
    print_layout_2d(R)
    #  (6, 2):(8, 2) o (4, 3):(3, 1)
    # -> (6, 2):(8, 2) o (4, 3) -> (2, 2):(24, 2)  # 6/3 -> 2   8*3=24,  (2, 2) % 4 == (2, 2)
    # -> (6, 2):(8, 2) o (3, 1) -> (3, 1):(8, 2)   # 6/1 -> 6   8*1=8,   (6, 2) % 3 = (3, 1)
    # -> ((2, 2), (3,1)):((24,2), (8,2))   # 最后的 2可以被合并
composition_example()

Layout: (6,2):(8,2)
  0   2 
  8  10 
 16  18 
 24  26 
 32  34 
 40  42 

Layout: (4,3):(3,1)
  0   1   2 
  3   4   5 
  6   7   8 
  9  10  11 

Layout: ((2,2),3):((24,2),8)
  0   8  16 
 24  32  40 
  2  10  18 
 26  34  42 



In [ ]:
@cute.jit
def composition_example():
    """
    Demonstrates basic layout composition R = A ◦ B
    """
    A = cute.make_layout(20, stride=2) # Dynamic stride
    B = cute.make_layout((5, 4), stride=(4, 1))
    R = cute.composition(A, B)

    # print_layout_2d(A) 
    print_layout_2d(B)
    print_layout_2d(R) # (5,4):(8,2)
    # 20:2 o (5, 4):(4, 1)
    # 20:2 o (5, 4) -> 5:8
    # 20:2 o (4, 1) -> 4:2
    
    # (5, 4):(8, 2)
composition_example()

Layout: (5,4):(4,1)
  0   1   2   3 
  4   5   6   7 
  8   9  10  11 
 12  13  14  15 
 16  17  18  19 

Layout: (5,4):(8,2)
  0   2   4   6 
  8  10  12  14 
 16  18  20  22 
 24  26  28  30 
 32  34  36  38 



In [ ]:
@cute.jit
def composition_example():
    """
    Demonstrates basic layout composition R = A ◦ B
    """
    A = cute.make_layout((10, 2), stride=(16, 4)) # Dynamic stride
    B = cute.make_layout((5, 4), stride=(1, 5))
    R = cute.composition(A, B)
    print_layout_2d(A)  # (10,2):(16,4)  o  (5,4):(1,5)
    print_layout_2d(B)
    print_layout_2d(R) #  (5,(2,2)):(16,(80,4))
    # (10,2):(16,4) o (5,4):(1,5)
    # (10,2):(16,4) o (5,1) -> 5:16
    # (10,2):(16,4) o (4,1) -> (2, 2):(80, 4)
    # (5,(2,2)):(16,(80,4))
    # 
    
composition_example()

DSLRuntimeError: DSLRuntimeError: `range_constexpr` requires constexpr (compile-time constant) for all arguments.
[92m💡 Suggestions:[0m
 Use `range` instead of `range_constexpr`.

In [ ]:
@cute.jit
def composition_example():
    """
    Demonstrates basic layout composition R = A ◦ B
    """
    A = cute.make_layout((12, (4,8)), stride=(59, (13,1))) # Dynamic stride
    B = cute.make_layout((3, 8), stride=(4, 2))
    R0 = cute.composition(A[0], B[0])
    R1 = cute.composition(A[1], B[1])
    cute.printf("{} {}", R0, R0[0].shape)
    cute.printf("{}, {} {}", R1, R1[0], R1[1])
    # cute.product(R0, R1)
    # R = cute.make_layout((R0.shape, R1.shape), (R0.stride, R1.stride))
    # (12, 59) o (3, 4) -> (3, 236)
    # (4, 8):(13, 1) o 8:2
    # (4, 8):(13, 1) o 8:2 -> (2, 4):(26,1)
    
    
composition_example()

3:236 3
(2,4):(26,1), 2:26 4:1


In [ ]:
@cute.jit
def print_layout_3d(L: cute.Tensor):
    m, n, g = L.shape
    m = cute.size(L, [0])
    n = cute.size(L, [1])
    cute.printf("Layout: {}", L)
    for i in cute.range_constexpr(m):
        for k in cute.range_constexpr(g):
            for j in cute.range_constexpr(n):
                cute.printf("%3d ", L((i, j, k)), end="")
        cute.printf("")
    cute.printf("")
@cute.jit
def divide_1d_example(): 
    A = cute.make_layout((4,2,3), stride = (2,1,8))
    B = cute.make_layout(4, stride = 2)
    ret = cute.logical_divide(A, B)
    print_layout_3d(A)
    # cute.printf(ret)
    print_layout_2d(ret)
divide_1d_example()

Layout: (4,2,3):(2,1,8)
  0   1   8   9  16  17 
  2   3  10  11  18  19 
  4   5  12  13  20  21 
  6   7  14  15  22  23 

Layout: ((2,2),(2,3)):((4,1),(2,8))
  0   2   8  10  16  18 
  4   6  12  14  20  22 
  1   3   9  11  17  19 
  5   7  13  15  21  23 



In [ ]:
@cute.jit
def divide_2d_example(): 
    A = cute.make_layout((9, (4,8)), stride=(59, (13,1)))
    B = cute.make_layout((3, (2,4)), stride=(3, (1,8)))
    C = cute.logical_divide(A, (cute.make_layout(3, stride=3), cute.make_layout((2,4), stride=(1,8))))
    print_layout_2d(A)
    
    print_layout_2d(C)
    cute.printf(C)
divide_2d_example()

@cute.jit
def divide_2d_example_simple(): 
    # change the stride to left
    A = cute.make_layout((9, (4,8)))
    B = cute.make_layout((3, (2,4)), stride=(3, (1,8)))
    C = cute.logical_divide(A, (cute.make_layout(3, stride=3), cute.make_layout((2,4), stride=(1,8))))
    print_layout_2d(A)
    
    print_layout_2d(C)
    cute.printf(C)
divide_2d_example_simple()

Layout: (9,(4,8)):(59,(13,1))
  0  13  26  39   1  14  27  40   2  15  28  41   3  16  29  42   4  17  30  43   5  18  31  44   6  19  32  45   7  20  33  46 
 59  72  85  98  60  73  86  99  61  74  87 100  62  75  88 101  63  76  89 102  64  77  90 103  65  78  91 104  66  79  92 105 
118 131 144 157 119 132 145 158 120 133 146 159 121 134 147 160 122 135 148 161 123 136 149 162 124 137 150 163 125 138 151 164 
177 190 203 216 178 191 204 217 179 192 205 218 180 193 206 219 181 194 207 220 182 195 208 221 183 196 209 222 184 197 210 223 
236 249 262 275 237 250 263 276 238 251 264 277 239 252 265 278 240 253 266 279 241 254 267 280 242 255 268 281 243 256 269 282 
295 308 321 334 296 309 322 335 297 310 323 336 298 311 324 337 299 312 325 338 300 313 326 339 301 314 327 340 302 315 328 341 
354 367 380 393 355 368 381 394 356 369 382 395 357 370 383 396 358 371 384 397 359 372 385 398 360 373 386 399 361 374 387 400 
413 426 439 452 414 427 440 453 415 428 441 454 416 429 442 455 417

In [ ]:
@cute.jit
def divide_2d_example_simple(): 
    # change the stride to left
    A = cute.make_layout((9, (4,8)))
    tiler = (cute.make_layout(3, stride=3), cute.make_layout((2,4), stride=(1,8)))
    B = cute.logical_divide(A, tiler)
    C = cute.zipped_divide(A, tiler)
    cute.printf(B)
    cute.printf(C)

    print_layout_2d(A)
    print_layout_2d(B)
    
    print_layout_2d(C)
    cute.printf("{}", C((0,3)))
divide_2d_example_simple()

((3,3),((2,4),4)):((3,1),((9,72),18))
((3,(2,4)),(3,4)):((3,(9,72)),(1,18))
Layout: (9,(4,8)):(1,(9,36))
  0   9  18  27  36  45  54  63  72  81  90  99 108 117 126 135 144 153 162 171 180 189 198 207 216 225 234 243 252 261 270 279 
  1  10  19  28  37  46  55  64  73  82  91 100 109 118 127 136 145 154 163 172 181 190 199 208 217 226 235 244 253 262 271 280 
  2  11  20  29  38  47  56  65  74  83  92 101 110 119 128 137 146 155 164 173 182 191 200 209 218 227 236 245 254 263 272 281 
  3  12  21  30  39  48  57  66  75  84  93 102 111 120 129 138 147 156 165 174 183 192 201 210 219 228 237 246 255 264 273 282 
  4  13  22  31  40  49  58  67  76  85  94 103 112 121 130 139 148 157 166 175 184 193 202 211 220 229 238 247 256 265 274 283 
  5  14  23  32  41  50  59  68  77  86  95 104 113 122 131 140 149 158 167 176 185 194 203 212 221 230 239 248 257 266 275 284 
  6  15  24  33  42  51  60  69  78  87  96 105 114 123 132 141 150 159 168 177 186 195 204 213 222 231 240 249 258 267 2

In [ ]:
@cute.jit
def print_layout_3d(L: cute.Tensor):
    m, n, g = L.shape
    m = cute.size(L, [0])
    n = cute.size(L, [1])
    cute.printf("Layout: {}", L)
    for i in cute.range_constexpr(m):
        for k in cute.range_constexpr(g):
            for j in cute.range_constexpr(n):
                cute.printf("%3d ", L((i, j, k)), end="")
        cute.printf("")
    cute.printf("")
@cute.jit
def print_layout_1d(L: cute.Tensor):
    cute.printf("Layout: {}", L)
    for i in cute.range(L.shape):
        cute.printf("%3d", L(i), end = "")
    cute.printf("")
@cute.jit
def product_1d(): 
    A = cute.make_layout((2,2), stride=(4,1))
    B = cute.make_layout(6, stride=1)
    C = cute.logical_product(A,B)
    cute.printf(C)
    # print_layout_2d(A)
    # print_layout_1d(B)
    # print_layout_2d(C)
product_1d()
@cute.jit
def product_2d(): 
    A = cute.make_layout((2,5), stride=(5,1))
    B = cute.make_layout((3,5), stride=(4,6))
    C = cute.logical_product(A,B)
    cute.printf(C)
    print_layout_2d(A)
    print_layout_2d(B)
    print_layout_2d(C)
product_2d()

((2,2),(2,3)):((4,1),(2,8))
((2,5),(3,5)):((5,1),(40,60))
Layout: (2,5):(5,1)
  0   1   2   3   4 
  5   6   7   8   9 

Layout: (3,5):(4,6)
  0   6  12  18  24 
  4  10  16  22  28 
  8  14  20  26  32 

Layout: ((2,5),(3,5)):((5,1),(40,60))
  0  40  80  60 100 140 120 160 200 180 220 260 240 280 320 
  5  45  85  65 105 145 125 165 205 185 225 265 245 285 325 
  1  41  81  61 101 141 121 161 201 181 221 261 241 281 321 
  6  46  86  66 106 146 126 166 206 186 226 266 246 286 326 
  2  42  82  62 102 142 122 162 202 182 222 262 242 282 322 
  7  47  87  67 107 147 127 167 207 187 227 267 247 287 327 
  3  43  83  63 103 143 123 163 203 183 223 263 243 283 323 
  8  48  88  68 108 148 128 168 208 188 228 268 248 288 328 
  4  44  84  64 104 144 124 164 204 184 224 264 244 284 324 
  9  49  89  69 109 149 129 169 209 189 229 269 249 289 329 



In [ ]:
@cute.jit
def product_2d(): 
    A = cute.make_layout((2,5), stride=(5,1))
    B = cute.make_layout((3,4), stride=(1,3))
    C = cute.blocked_product(A,B)
    cute.printf(C)
    cute.printf(cute.coalesce(C))
    print_layout_2d(A)
    print_layout_2d(B)
    print_layout_2d(C)
product_2d()

((2,3),(5,4)):((5,10),(1,30))
(6,5,4):(5,1,30)
Layout: (2,5):(5,1)
  0   1   2   3   4 
  5   6   7   8   9 

Layout: (3,4):(1,3)
  0   3   6   9 
  1   4   7  10 
  2   5   8  11 

Layout: ((2,3),(5,4)):((5,10),(1,30))
  0   1   2   3   4  30  31  32  33  34  60  61  62  63  64  90  91  92  93  94 
  5   6   7   8   9  35  36  37  38  39  65  66  67  68  69  95  96  97  98  99 
 10  11  12  13  14  40  41  42  43  44  70  71  72  73  74 100 101 102 103 104 
 15  16  17  18  19  45  46  47  48  49  75  76  77  78  79 105 106 107 108 109 
 20  21  22  23  24  50  51  52  53  54  80  81  82  83  84 110 111 112 113 114 
 25  26  27  28  29  55  56  57  58  59  85  86  87  88  89 115 116 117 118 119 



In [ ]:
@cute.jit
def product_2d(): 
    A = cute.make_layout((2,5), stride=(5,1))
    B = cute.make_layout((3,4), stride=(1,3))
    C = cute.raked_product(A,B)
    cute.printf(C)
    cute.printf(cute.coalesce(C))
    print_layout_2d(A)
    print_layout_2d(B)
    print_layout_2d(C)
product_2d()

((3,2),(4,5)):((10,5),(30,1))
(3,2,4,5):(10,5,30,1)
Layout: (2,5):(5,1)
  0   1   2   3   4 
  5   6   7   8   9 

Layout: (3,4):(1,3)
  0   3   6   9 
  1   4   7  10 
  2   5   8  11 

Layout: ((3,2),(4,5)):((10,5),(30,1))
  0  30  60  90   1  31  61  91   2  32  62  92   3  33  63  93   4  34  64  94 
 10  40  70 100  11  41  71 101  12  42  72 102  13  43  73 103  14  44  74 104 
 20  50  80 110  21  51  81 111  22  52  82 112  23  53  83 113  24  54  84 114 
  5  35  65  95   6  36  66  96   7  37  67  97   8  38  68  98   9  39  69  99 
 15  45  75 105  16  46  76 106  17  47  77 107  18  48  78 108  19  49  79 109 
 25  55  85 115  26  56  86 116  27  57  87 117  28  58  88 118  29  59  89 119 



In [ ]:
@cute.jit
def zipped_partition():
    A = cute.make_layout((4,12)) 
    print_layout_2d(A)
    
    tiler = (cute.make_layout(2), cute.make_layout(6))
    
    tile_a = cute.zipped_divide(A, tiler)
    print_layout_2d(tile_a)
    cute.printf(tile_a[(None, (2, 2))])
    # print_layout_2d(tile_a[(None, (0,0))])
    
    
zipped_partition()

DSLRuntimeError: DSLRuntimeError: 💥💥💥 Error during runtime code generation for function `zipped_partition` 💥💥💥

In [46]:
@cute.kernel
def vectorized_elementwise_add_kernel(
    gA: cute.Tensor,
):

    m, n = gA.shape[1]

    a_val = gA[(None, (mi, ni))].load()
    b_val = gB[(None, (mi, ni))].load()
    print(f"[DSL INFO] sliced gA = {gA[(None, (mi, ni))]}")
    print(f"[DSL INFO] sliced gB = {gB[(None, (mi, ni))]}")

    # Perform element-wise addition
    gC[(None, (mi, ni))] = a_val + b_val
@cute.jit
def vectorized_elementwise_add(
    mA: cute.Tensor,
):
    threads_per_block = 256

    gA = cute.zipped_divide(mA, (1, 4))

    print(f"[DSL INFO] Tiled Tensors:")
    print(f"[DSL INFO]   gA = {gA}")
    ret = gA.load()
    cute.printf(ret)
    # print_layout_2d(gA.layout)
    # cute.printf(gA[(None, (0,0))].load())
    # cute.printf(gA[(None, (0,1))])
    # cute.printf(gA)
    # cute.printf(gA.layout[(None, (0,1))])
    # vectorized_elementwise_add_kernel(gA).launch(
    #     grid=(1, 1, 1),
    #     block=(1, 1, 1),
    # )


a = torch.range(0, 47, dtype=torch.float32).reshape((4,12))

a_ = from_dlpack(a, assumed_align=1)

# compiled_func = cute.compile(vectorized_elementwise_add, a_)
vectorized_elementwise_add(a_)

# verify correctness
# torch.testing.assert_close(c, a + b

[DSL INFO] Tiled Tensors:
[DSL INFO]   gA = tensor<ptr<f32, generic, align<1>> o ((1,4),(4,3)):((0,1),(12,4))>


/tmp/ipykernel_1842782/1761227168.py:38: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  a = torch.range(0, 47, dtype=torch.float32).reshape((4,12))


DSLRuntimeError: DSLRuntimeError: 🧊🧊🧊 ICE IR Verification Failed 🧊🧊🧊
  Caused exception: Verification failed:
error: "vectorized_elementwise_add"("/tmp/ipykernel_1842782/1761227168.py":26:0): 'cute.print' op operand #0 must be variadic of Type interface for cute types can be printed by `cute.print`, but got 'vector<48xf32>'
 note: "vectorized_elementwise_add"("/tmp/ipykernel_1842782/1761227168.py":26:0): see current operation: "cute.print"(%19) <{fmt = "{}\0A"}> : (vector<48xf32>) -> ()

In [ ]:
cute.make_fragment()
